# ENTREGABLE 4

# INSTRUCCIONES

Utilizar el archivo CSV (`dataset_banco_clean.csv`) con 45189 filas y 17 columnas y aplicar las técnicas de normalización del entregable 3.

In [1]:
# imports
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("/content/dataset_banco_clean.csv")
print(df.head())

   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no   2143.0     yes   no   
1   44    technician   single  secondary      no     29.0     yes   no   
2   33  entrepreneur  married  secondary      no      2.0     yes  yes   
3   47   blue-collar  married    unknown      no   1506.0     yes   no   
4   33       unknown   single    unknown      no      1.0      no   no   

   contact  day month  duration  campaign  pdays  previous poutcome   y  
0  unknown    5   may     261.0         1   -1.0         0  unknown  no  
1  unknown    5   may     151.0         1   -1.0         0  unknown  no  
2  unknown    5   may      76.0         1   -1.0         0  unknown  no  
3  unknown    5   may      92.0         1   -1.0         0  unknown  no  
4  unknown    5   may     198.0         1   -1.0         0  unknown  no  


# Objetivo

Generar un model de clasificación capaz de predecir la clase de flor en función de las carácterísticas del dataset

* Aplicar las técnicas oportunas de procesamiento de datos

* Generar split de los datos

* Valorar diferentes modelos de clasificación

* Comparación entre modelos

* Ensemble

* Métricas

* Conclusiones finales

In [10]:
# Paso 1: Preprocesamiento de datos
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [11]:
# Manejo de valores faltantes
numeric_features = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']
categorical_features = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [12]:
# Paso 2: División de datos
from sklearn.model_selection import train_test_split

X = df.drop(columns=['y'])
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Paso 3: Selección y entrenamiento de modelos
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

models = {
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC()
}

for name, model in models.items():
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', model)])
    clf.fit(X_train, y_train)
    print(f"{name} trained successfully.")

Decision Tree trained successfully.
Random Forest trained successfully.
SVM trained successfully.


In [15]:
# Paso 4: Evaluación de modelos
from sklearn.metrics import accuracy_score, classification_report

for name, model in models.items():
    y_pred = clf.predict(X_test)
    print(f"Evaluación de {name}:")
    print(f"Accuracy Score: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))

Evaluación de Decision Tree:
Accuracy Score: 0.9032971896437265
              precision    recall  f1-score   support

          no       0.92      0.98      0.95      7978
         yes       0.69      0.32      0.44      1060

    accuracy                           0.90      9038
   macro avg       0.80      0.65      0.69      9038
weighted avg       0.89      0.90      0.89      9038

Evaluación de Random Forest:
Accuracy Score: 0.9032971896437265
              precision    recall  f1-score   support

          no       0.92      0.98      0.95      7978
         yes       0.69      0.32      0.44      1060

    accuracy                           0.90      9038
   macro avg       0.80      0.65      0.69      9038
weighted avg       0.89      0.90      0.89      9038

Evaluación de SVM:
Accuracy Score: 0.9032971896437265
              precision    recall  f1-score   support

          no       0.92      0.98      0.95      7978
         yes       0.69      0.32      0.44      1060



El modelo elegido es random forest

In [ ]:
# El ensemble no es necesario ya que el Random Forest ya es un ensemble de árboles de decisión.

In [17]:
# Paso 7: Métricas finales
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Entrenar el modelo Random Forest
clf_rf = Pipeline(steps=[('preprocessor', preprocessor),
                         ('classifier', RandomForestClassifier())])
clf_rf.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_rf = clf_rf.predict(X_test)

# Calcular métricas finales
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, pos_label='yes')
recall_rf = recall_score(y_test, y_pred_rf, pos_label='yes')
f1_rf = f1_score(y_test, y_pred_rf, pos_label='yes')

print("Métricas finales para Random Forest:")
print(f"Accuracy: {accuracy_rf}")
print(f"Precisión: {precision_rf}")
print(f"Recall: {recall_rf}")
print(f"F1-score: {f1_rf}")


Métricas finales para Random Forest:
Accuracy: 0.9065058641292322
Precisión: 0.6759410801963993
Recall: 0.389622641509434
F1-score: 0.49431478156792336


# Conclusiones

Precisión: El modelo tiene una precisión del 67.59%. Esto significa que de todas las predicciones positivas que hizo el modelo, alrededor del 67.59% fueron correctas.
Recall: El recall, también conocido como sensibilidad, es del 38.96%. Esto indica que de todos los casos positivos en los datos reales, el modelo fue capaz de identificar correctamente alrededor del 38.96%.
F1-score: El F1-score, que es la media armónica de precisión y recall, es del 49.43%. Esta métrica es útil cuando queremos encontrar un equilibrio entre precisión y recall.
Exactitud: La exactitud general del modelo es del 90.65%, lo que indica que el 90.65% de todas las predicciones realizadas por el modelo fueron correctas.

Aunque el modelo tiene una precisión relativamente alta, su recall es más bajo, lo que sugiere que puede estar perdiendo algunos casos positivos. Esto podría indicar que el modelo tiene dificultades para identificar ciertos patrones en los datos. Dependiendo del contexto y de las necesidades específicas del problema, puede ser necesario ajustar el modelo o explorar otras técnicas para mejorar su rendimiento.